## Flottants

Un nombre flottant est formé de trois éléments : la mantisse, l'exposant et le signe. Le bit de poids fort est le bit de signe : si ce bit est à 1, le nombre est négatif, et s’il est à 0, le nombre est positif. Les e bits suivants représentent l'exposant décalé, et les m bits suivants (m bits de poids faible) représentent la mantisse.

### Format général flottants IEEE 754

| Signe| Exposant décalé| Mantisse|
|:------------:|:---------------------:|:-------------:|
| (1 bit)| (e bits)| (m bits)|
 	
    
### Décalage de l'exposant

L'exposant peut être positif ou négatif. Afin de garantir la compatibilité des comparaisons entre les entiers naturels et entre les flottants on n'utilise pas le C2. Ici l'exposant est **décalé**, afin de le stocker sous forme d'un nombre non signé. Ce décalage est de $2^{e−1} − 1$ ($e$ représente le nombre de bits de l'exposant) ; il s'agit donc d'une valeur constante une fois que le nombre de bits $e$ est fixé.

L'interprétation d'un nombre (autre qu'infini) est donc :

valeur = signe × mantisse × 2(exposant − décalage)

avec

signe = ±1

decalage = $2^{e−1} − 1$ 

### Exceptions

Dans le cas général (l'exposant est différent de 0 et de $2^e − 1$), le bit de poids fort de la mantisse est toujours égal à 1. De ce fait il est inutile de le représenter explicitement.  Le nombre est dit « normalisé ». Si l'exposant décalé est nul, le bit de poids fort de la mantisse est nul, et le nombre est 'dé-normalisé'. 

Il y a trois cas particuliers :

- si l'exposant décalé et la mantisse sont tous deux nuls, le nombre est ±0 (selon le bit de signe)
- si l'exposant décalé est égal à $ 2^{e} - 1$, et si la mantisse est nulle, le nombre est ±$\inf$ (selon le bit de signe)
- si l'exposant décalé est égal à $ 2^{e} - 1$, mais que la mantisse n'est pas nulle, le nombre est NaN (not a number : pas un nombre).

Type| 	Exposant décalé| 	Mantisse
:------------:|:---------------------:|:-------------:
Zéros| 	0| 	0
Nombres dénormalisés |	0 |	différente de 0
Nombres normalisés 	| 1 à $2^e-2 $|	quelconque
Infinis |	$ 2^e-1 $|	0
NaNs |$ 2^e-1 $|	différente de 0 


La norme IEEE754-2008 prévoit 3 types de configurations en fonction du nombre de bits disponibles pour représenter les flottants:

 `     `|Simple précision|Double précision|Quadruple précision
:---------------------:|:------------:|:---------------------:|:-------------:
Bit de signe|1|1|1
Bits de l’exposant|8|11|15
Bits de la mantisse|23|52|112
Nombre total de bits|32|64|128
Codage de l’exposant|Excédant 127|Excédant 1023|Excédant 16383
Variation de l’exposant|-126 à +127|-1022 à +1023|-16382 à 16383
Plus petit nombre normalisé|$2^{-126}$|$2^{-1022}$|$2^{-16382}$
Plus grand nombre normalisé|$2^{+128}$|$2^{+1024}$|$2^{+16384}$
Échelle des nombres décimaux|environ $10^{–38}$ à $10^{+38}$|$10^{-308}$ à $10^{+308}$|$10^{-4932}$ à $10^{+4932}$
Plus petit nombre non normalisé|environ $10^{-45}$|environ $10^{-324}$|



Exemple:

$1= (signe,exposant,mantisse)= (+,0,1.0)$ 

ce qui donne: $1=(0,0+2^{e−1} − 1,0)$

et en binaire si on représente tous les bits sur 64 bits: 

$1=(0,01111111111,0000000000000000000000000000000000000000000000000000)$

Ce qui est concaténé sur 64 bits:

$0011111111110000000000000000000000000000000000000000000000000000$

Mais c'est un brin illisible donc on représente cela sous forme hexadécimale:

$3FF0000000000000$

Autre exemple: 


|Base 10|Base 16|Base 2|
|-----|-----|-----|
|3.14|40091eb851eb851f|0100000000001001000111101011100001010001111010111000010100011111|





Pour accéder à la représentation en mémoire d'un flottant, on le met sous la forme d'une séquence de bytes. La fonction "pack" du module struct (https://docs.python.org/3/library/struct.html) permet d'effectuer cette transformation, on note en particulier l'usage de "!" pour fixer une représentation gros-boutiste. D'autre part, les flottants en python sont représentés en double précision (binary64). La fonction unpack fait la transformation inverse.

La chaîne de caractères produite est préfixée par "b" et est la suite des codes ascii (ou de leur interprétation si elle est affichable) représentant les codes hexadécimaux des octets.


<img src="fig/1280px-USASCII_code_chart.png" alt="1280px-USASCII_code_chart.png"
	title="1280px-USASCII_code_chart.png" width="400"  />

Testez le code suivant avec les valeurs 1.0 , 3.14 et 13019830.68

Expliquez les valeurs que vous affichez.


Aide:
```python
from struct import pack,unpack

t=pack('!d',13019830.68)
print(t)  #d signifie que l'on utilise la représentation en double précision, f pour la simple précision

print("{:x}".format(t[0]))  # il est aussi possible de lire la première case du tableau retourné par unpack et contenant le début de la séquence de bytes 
```

"La suite des codes ascii (ou de leur interprétation si elle est affichable) représentant les codes hexadécimaux des octets"... Ce n'est guère lisible! Faites en sorte d'afficher directement en hexadécimal la séquence de bytes du flottant. Coder cette fonction, puis utilisez-là pour visualiser le format flottant pour  1.0 , 3.14 et 13019830.68

```Python
from struct import pack,unpack


def floattohexa(f):
    """Retourne une chaine représentant le flottant en hexadécimal"""
    t=pack('!d',f)
    s=""
    pass  # changer le code ici
    return s

print(floattohexa(3.14))
print(floattohexa(1.0))
print(floattohexa(13019830.68))
```

On souhaite maintenant pouvoir obtenir le triplet (signe, exposant, mantisse) décrivant un flottant donné. Pour ce faire, on crée une nouvelle fonction sur le modèle de ``` floattohexa```:

```Python
def floattobin(f):
    """Retourne une chaine représentant le flottant en binaire"""
    t=pack('!d',f)
    s=""
    pass  #changer le code ici
    return s
```

Puis on récupére les bits correspondants au signe, à l'exposant et à la mantisse et les convertir en entier.

```Python

def triplet(f):
    """
        Prend en argument un flottant en double précision et retourne le triplet:
        -s, le bit de signe
        -e, l'exposant sous forme d'entier
        -m, la mantisse en hexadécimal.
    
    """
    bytes=pack('!d',f)

    s=floattobin(f)
    return (......)


print(triplet(13019830.68))
print(triplet(1.0))
print(triplet(3.14))
```

On obtient:

```
(0, 23, '8d556d5c28f5c')
(0, 0, '0')
(0, 1, '91eb851eb851f')
```   

Écrire la fonction inverse dans le cas des flottants normalisés. Elle prend en entrée un triplet (s,e,m) et retourne le flottant correspondant. Il est possible de le faire en une seule expression.  Essayez la avec les triplets suivants. Que constatez-vous?

- (0, 23, '8d556d5c28f5c')
- (0, 0, '0')
- (0, 1, '91eb851eb851f')

```Python
from decimal import Decimal

def flottant(s,e,m):
    """Retourne le flottant en double précision décrit par :
        -son bit de signe s
        -son exposant e
        -sa mantisse m
    """
    return ......


print(Decimal(flottant(0, 23, '8d556d5c28f5c'))) 
print(Decimal(flottant(0, 0, '0'))) 
print(Decimal(flottant(0, 1, '91eb851eb851f'))) 

print(Decimal(flottant(*(triplet(1.0)))))  
print(Decimal(flottant(*(triplet(1.0)))))  
print(Decimal(flottant(*(triplet(1.0)))))  
```



Les valeurs des flottants sont (presque) toujours des approximations. Testez avec 0.1, 0.2, 0.3 

```Python
print(Decimal(0.1))
```

Mais c'est facile d'oublier cette aproximation:
```Python
print(0.1)
print(0.2)
```

Toutefois elle se rappelle facilement à nous... Essayez à présent d'additionner 0.1 et 0.2 et d'afficher le résultat.

La documentation de la primitive `round()` indique qu’elle arrondit à la valeur la plus proche en s’**éloignant de zéro** s'il y a équidistance entre deux valeurs. Comme expliquez-vous le comportement suivant?

In [8]:
print(round(2.675, 2))

2.67


Pour répondre il faut probablement afficher les valeurs repésentant 2.67,  2.675, 2.68

Une autre conséquence du fait que 0.1 n’est pas exactement stocké 1/10 est que la somme de dix valeurs de 0.1 ne donne pas 1.0 non plus. Additionnez $0.1$ dans une variable, 10 fois, et affichez le résultat. Le résultat n'est pas 1.0! 

Testez avec 10000000 tours et calculez l'écart final avec la valeur que l'on aurait dû trouver si la réprésentation était parfaite.

Mais dans d'autres langages, les choses peuvent être pires. Par exemple en Java on peut utiliser les flottants sur 32 bits. Il est possible d'utiliser ce type en Python avec la librairie numpy.

In [12]:
import numpy

sum = numpy.float32(0.0)
x = numpy.float32(0.1)
print(x)

0.1


Dans ce cas le flottant est codé non plus sur 64 bits mais seulement sur 32. Observez les conséquences sur l'exemple précédent.

## Cas d'école: 

Le 25 février 1991, un missile Patriote échoue à intercepter un missile Scud à Dharan.  Dans la batterie de missile, afin de calculer la trajectoire d'interception, un compteur compte les 1/10 de seconde. Puis ce nombre est multiplié par 0.1 qui est stocké dans un registre de 24 bits mais pas en virgule flottante. Ce nombre est stocké en virgule fixe comme ceci:

 `0.00011001100110011001100`

Pour transposer ce choix en virgule flottante, il faudrait n'utiliser qu'une faible portion de la capacité de représentation. Cela revient pour la représentation de 0.1 à remplacer: 

`(0, -4, '999999999999a')`

par:

`(0, -4, '9999800000000')`

Expliquez.

A l'aide de votre travail en Python affichez l'écart que cela représente. 

Tester ce que cela donne au bout de 100h (temps que la batterie a été mise en service alors).

Sachant qu'un missile Scud se déplace à environ 1 676 mètres par seconde, quel écart de distance cela donne-t-il? 
